In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization
import shap
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns
import time
import tensorflow as tf

# فعال کردن لاگ‌های دیباگ TensorFlow
tf.debugging.set_log_device_placement(True)

# تابع برای محاسبه معیارهای RMSE، R² و MAPE
def calculate_metrics(y_true, y_pred, metric_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    # محاسبه MAPE (در درصد)
    mask = y_true != 0  # فقط مقادیر غیرصفر را در نظر بگیر
    if np.sum(mask) > 0:
        mape = mean_absolute_percentage_error(y_true[mask], y_pred[mask]) * 100
    else:
        mape = np.nan  # در صورت وجود فقط صفرها، MAPE تعریف‌نشده است
    return {'Metric': metric_name, 'RMSE': rmse, 'R²': r2, 'MAPE': mape}

# تابع برای تحلیل بوت‌استرپ
def bootstrap_metrics(model, X_test, y_test, algo_name, n_iterations=100):
    bootstrap_results = []
    for _ in range(n_iterations):
        X_boot, y_boot = resample(X_test, y_test, random_state=np.random.randint(0, 10000))
        y_pred_boot = model.predict(X_boot) if algo_name != 'DNN' else model.predict(X_boot, batch_size=16, verbose=0)
        for i, metric in enumerate(['t1', 't2', 't3', 't4', 't5']):
            rmse = np.sqrt(mean_squared_error(y_boot[metric], y_pred_boot[:, i]))
            r2 = r2_score(y_boot[metric], y_pred_boot[:, i])
            mask = y_boot[metric] != 0
            if np.sum(mask) > 0:
                mape = mean_absolute_percentage_error(y_boot[metric][mask], y_pred_boot[:, i][mask]) * 100
            else:
                mape = np.nan
            bootstrap_results.append({'Metric': metric, 'RMSE': rmse, 'R²': r2, 'MAPE': mape})
    return pd.DataFrame(bootstrap_results)

# تابع برای جمع‌بندی نتایج بوت‌استرپ
def summarize_bootstrap(bootstrap_df, algo_name, year):
    summary = bootstrap_df.groupby('Metric').agg({
        'RMSE': ['mean', lambda x: np.percentile(x, 2.5), lambda x: np.percentile(x, 97.5)],
        'R²': ['mean', lambda x: np.percentile(x, 2.5), lambda x: np.percentile(x, 97.5)],
        'MAPE': ['mean', lambda x: np.percentile(x, 2.5), lambda x: np.percentile(x, 97.5)]
    }).round(2)
    summary = summary.reset_index()
    summary['Algorithm'] = algo_name
    summary['Year'] = year
    return summary

# تابع برای حذف داده‌های پرت
def remove_outliers(df, column, iqr_factor=1.5):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - iqr_factor * IQR
    upper_bound = Q3 + iqr_factor * IQR
    df_filtered = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df_filtered

# تعریف مسیرهای دایرکتوری
base_dir = "D:\\UPM\\Machine Learning"
current_dir = os.path.join(base_dir, "Scenario 1 Present Climate")
dir_2050 = os.path.join(base_dir, "Scenario 2 2050")
dir_2080 = os.path.join(base_dir, "Scenario 2 2080")

# مسیر فایل‌های داده
current_file = os.path.join(current_dir, 'Synthetic_data_sdv_dummies_1826_Current Climate.csv')
file_2050 = os.path.join(dir_2050, 'Synthetic_data_future_2050_processed.csv')
file_2080 = os.path.join(dir_2080, 'Synthetic_data_future_2080_processed.csv')

# بارگذاری داده‌ها با مدیریت خطا
try:
    df_current = pd.read_csv(current_file)
    df_2050 = pd.read_csv(file_2050)
    df_2080 = pd.read_csv(file_2080)
    print("All datasets loaded successfully!")
except FileNotFoundError as e:
    print(f"Error: File not found: {e}")
    print("Files in current directory:", os.listdir(current_dir))
    print("Files in 2050 directory:", os.listdir(dir_2050))
    print("Files in 2080 directory:", os.listdir(dir_2080))
    raise
except Exception as e:
    print(f"Unexpected error: {e}")
    raise

# حذف داده‌های پرت برای همه داده‌ها
for target in ['t1', 't2', 't3', 't4', 't5']:
    df_current = remove_outliers(df_current, target, iqr_factor=2.0 if target == 't1' else 1.5)
    df_2050 = remove_outliers(df_2050, target)
    df_2080 = remove_outliers(df_2080, target)

# بررسی ستون‌ها
print("Columns in df_current:", df_current.columns.tolist())
print("Columns in df_2050:", df_2050.columns.tolist())
print("Columns in df_2080:", df_2080.columns.tolist())

# هم‌تراز کردن ستون‌ها
reference_columns = df_current.drop(columns=['t1', 't2', 't3', 't4', 't5']).columns
df_2050 = df_2050.reindex(columns=reference_columns.union(['t1', 't2', 't3', 't4', 't5']), fill_value=0)
df_2080 = df_2080.reindex(columns=reference_columns.union(['t1', 't2', 't3', 't4', 't5']), fill_value=0)

# بررسی ستون‌های گمشده یا اضافی
print("Missing columns in df_2050:", set(reference_columns) - set(df_2050.columns))
print("Extra columns in df_2050:", set(df_2050.columns) - set(reference_columns.union(['t1', 't2', 't3', 't4', 't5'])))
print("Missing columns in df_2080:", set(reference_columns) - set(df_2080.columns))
print("Extra columns in df_2080:", set(df_2080.columns) - set(reference_columns.union(['t1', 't2', 't3', 't4', 't5'])))

# تابع پیش‌پردازش داده‌ها
def preprocess_data(df, reference_columns=None):
    df_encoded = df.copy()
    X = df_encoded.drop(columns=['t1', 't2', 't3', 't4', 't5'])
    y = df_encoded[['t1', 't2', 't3', 't4', 't5']]
    if reference_columns is not None:
        X = X.reindex(columns=reference_columns, fill_value=0)
    X_scaled = scaler.fit_transform(X) if df is df_current else scaler.transform(X)
    return X_scaled, y, X.columns

# مقداردهی اولیه اسکالر و پیش‌پردازش داده‌ها
scaler = StandardScaler()
reference_columns = df_current.drop(columns=['t1', 't2', 't3', 't4', 't5']).columns
X_current, y_current, feature_names = preprocess_data(df_current, reference_columns)
X_2050, y_2050, _ = preprocess_data(df_2050, reference_columns)
X_2080, y_2080, _ = preprocess_data(df_2080, reference_columns)

# بررسی ستون‌های dummy مورد انتظار
expected_dummies = {
    'P10': ['P10_RoofwithInsulationBatt244mm'],
    'P11': ['P11_QuadrupleLoEFilms(88)3mm/8mmKrypton'],
    'P23': ['P23_FloorwithInsulationBatt244mm'],
    'P24': ['P24_TrpLoEFilm(66)Clr6mm/13mmAir'],
    'P25': ['P25_VRF-DOAS']
}
for var, dummies in expected_dummies.items():
    for dummy in dummies:
        if dummy not in df_2050.columns:
            print(f"Warning: {dummy} not found in df_2050")
        if dummy not in df_2080.columns:
            print(f"Warning: {dummy} not found in df_2080")

# تقسیم داده‌ها به مجموعه‌های آموزشی و آزمایشی
X_train_current, X_test_current, y_train_current, y_test_current = train_test_split(
    X_current, y_current, test_size=0.4, random_state=42
)
X_train_2050, X_test_2050, y_train_2050, y_test_2050 = train_test_split(
    X_2050, y_2050, test_size=0.4, random_state=42
)
X_train_2080, X_test_2080, y_train_2080, y_test_2080 = train_test_split(
    X_2080, y_2080, test_size=0.4, random_state=42
)

# مقداردهی اولیه مدل‌های پیش‌فرض
rf_default = RandomForestRegressor(random_state=42)
xgb_default = XGBRegressor(random_state=42)
dnn_default = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_current.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(5)
])
dnn_default.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())

# آموزش مدل‌های پیش‌فرض با EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
rf_default.fit(X_train_current, y_train_current)
xgb_default.fit(X_train_current, y_train_current)
dnn_default.fit(X_train_current, y_train_current, epochs=50, batch_size=16, validation_split=0.2, callbacks=[early_stopping], verbose=1)

# ذخیره مدل DNN پیش‌فرض
output_dir = os.path.join(base_dir, "Outputs")
os.makedirs(output_dir, exist_ok=True)
dnn_default.save(os.path.join(output_dir, 'dnn_current_default.h5'))

# DNN بدون Transfer Learning برای 2050
dnn_no_transfer_2050 = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_2050.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(5)
])
dnn_no_transfer_2050.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())
dnn_no_transfer_2050.fit(X_train_2050, y_train_2050, epochs=50, batch_size=16, validation_split=0.2, callbacks=[early_stopping], verbose=1)

# DNN بدون Transfer Learning برای 2080
dnn_no_transfer_2080 = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_2080.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(5)
])
dnn_no_transfer_2080.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())
dnn_no_transfer_2080.fit(X_train_2080, y_train_2080, epochs=50, batch_size=16, validation_split=0.2, callbacks=[early_stopping], verbose=1)

# تابع بهینه‌سازی هایپرپارامترهای Random Forest
def optimize_rf(X_train, y_train):
    start_time = time.time()
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5]
    }
    grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=3, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    grid_time = time.time() - start_time
    rf_grid = grid_search.best_estimator_

    start_time = time.time()
    def rf_evaluate(n_estimators, max_depth, min_samples_split):
        model = RandomForestRegressor(
            n_estimators=int(n_estimators), max_depth=int(max_depth) if max_depth > 0 else None,
            min_samples_split=int(min_samples_split), random_state=42
        )
        model.fit(X_train, y_train)
        return -np.mean([mean_squared_error(y_train.iloc[:, i], model.predict(X_train)[:, i]) for i in range(5)])
    
    pbounds = {'n_estimators': (100, 300), 'max_depth': (10, 30), 'min_samples_split': (2, 10)}
    optimizer = BayesianOptimization(f=rf_evaluate, pbounds=pbounds, random_state=42)
    optimizer.maximize(init_points=5, n_iter=10)
    bayes_time = time.time() - start_time
    rf_bayes = RandomForestRegressor(
        n_estimators=int(optimizer.max['params']['n_estimators']),
        max_depth=int(optimizer.max['params']['max_depth']) if optimizer.max['params']['max_depth'] > 0 else None,
        min_samples_split=int(optimizer.max['params']['min_samples_split']),
        random_state=42
    )
    rf_bayes.fit(X_train, y_train)
    return rf_grid, rf_bayes, grid_time, bayes_time

# تابع بهینه‌سازی هایپرپارامترهای XGBoost
def optimize_xgb(X_train, y_train):
    start_time = time.time()
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [3, 6],
        'learning_rate': [0.01, 0.1]
    }
    grid_search = GridSearchCV(XGBRegressor(random_state=42), param_grid, cv=3, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    grid_time = time.time() - start_time
    xgb_grid = grid_search.best_estimator_

    start_time = time.time()
    def xgb_evaluate(n_estimators, max_depth, learning_rate):
        model = XGBRegressor(
            n_estimators=int(n_estimators), max_depth=int(max_depth),
            learning_rate=learning_rate, random_state=42
        )
        model.fit(X_train, y_train)
        return -np.mean([mean_squared_error(y_train.iloc[:, i], model.predict(X_train)[:, i]) for i in range(5)])
    
    pbounds = {'n_estimators': (100, 300), 'max_depth': (3, 10), 'learning_rate': (0.01, 0.3)}
    optimizer = BayesianOptimization(f=xgb_evaluate, pbounds=pbounds, random_state=42)
    optimizer.maximize(init_points=5, n_iter=10)
    bayes_time = time.time() - start_time
    xgb_bayes = XGBRegressor(
        n_estimators=int(optimizer.max['params']['n_estimators']),
        max_depth=int(optimizer.max['params']['max_depth']),
        learning_rate=optimizer.max['params']['learning_rate'],
        random_state=42
    )
    xgb_bayes.fit(X_train, y_train)
    return xgb_grid, xgb_bayes, grid_time, bayes_time

# تابع بهینه‌سازی هایپرپارامترهای DNN
def optimize_dnn(X_train, y_train):
    start_time = time.time()
    best_score = float('inf')
    best_model = None
    for units in [(64, 32), (128, 64)]:
        for lr in [0.001, 0.01]:
            model = Sequential([
                Dense(units[0], activation='relu', input_shape=(X_train.shape[1],)),
                Dense(units[1], activation='relu'),
                Dense(5)
            ])
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss=tf.keras.losses.MeanSquaredError())
            model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, callbacks=[early_stopping], verbose=1)
            score = np.mean([mean_squared_error(y_train.iloc[:, i], model.predict(X_train, batch_size=16, verbose=0)[:, i]) for i in range(5)])
            if score < best_score:
                best_score = score
                best_model = model
    grid_time = time.time() - start_time
    dnn_grid = best_model

    start_time = time.time()
    def dnn_evaluate(units1, units2, learning_rate):
        model = Sequential([
            Dense(int(units1), activation='relu', input_shape=(X_train.shape[1],)),
            Dense(int(units2), activation='relu'),
            Dense(5)
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss=tf.keras.losses.MeanSquaredError())
        model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, callbacks=[early_stopping], verbose=1)
        return -np.mean([mean_squared_error(y_train.iloc[:, i], model.predict(X_train, batch_size=16, verbose=0)[:, i]) for i in range(5)])
    
    pbounds = {'units1': (32, 128), 'units2': (16, 64), 'learning_rate': (0.001, 0.1)}
    optimizer = BayesianOptimization(f=dnn_evaluate, pbounds=pbounds, random_state=42)
    optimizer.maximize(init_points=5, n_iter=10)
    bayes_time = time.time() - start_time
    dnn_bayes = Sequential([
        Dense(int(optimizer.max['params']['units1']), activation='relu', input_shape=(X_train.shape[1],)),
        Dense(int(optimizer.max['params']['units2']), activation='relu'),
        Dense(5)
    ])
    dnn_bayes.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=optimizer.max['params']['learning_rate']), loss=tf.keras.losses.MeanSquaredError())
    dnn_bayes.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, callbacks=[early_stopping], verbose=1)
    return dnn_grid, dnn_bayes, grid_time, bayes_time

# تابع تحلیل SHAP
def run_shap_analysis(model, X_test, feature_columns, model_name, year):
    print(f"SHAP analysis for {model_name} - Year {year}: X_test shape = {X_test.shape}, Features = {len(feature_columns)}")
    
    if X_test.shape[0] < 2:
        print(f"Skipping SHAP analysis for {model_name} - Year {year}: Too few samples ({X_test.shape[0]})")
        return pd.DataFrame()

    n_samples = min(10, X_test.shape[0])
    X_test_subset = X_test[:n_samples]
    if model_name.startswith('Random Forest') or model_name.startswith('XGBoost'):
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_test_subset)
    else:
        explainer = shap.KernelExplainer(model.predict, X_test_subset)
        shap_values = explainer.shap_values(X_test_subset)

    if isinstance(shap_values, list):
        shap_values = np.array(shap_values)
        shap_values = np.abs(shap_values).mean(axis=0)
    else:
        shap_values = np.abs(shap_values)

    shap_mean = shap_values.mean(axis=0)
    print(f"SHAP mean shape for {model_name} - Year {year}: {shap_mean.shape}")
    if shap_mean.ndim > 1:
        shap_mean = shap_mean.mean(axis=1)

    try:
        plt.figure(figsize=(10, 6))
        shap.summary_plot(shap_values, X_test_subset, feature_names=feature_columns, show=False)
        plt.title(f"SHAP Summary Plot for {model_name} - Year {year}")
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"shap_summary_{model_name.lower().replace(' ', '_')}_{year}.png"))
        plt.close('all')
    except Exception as e:
        print(f"Error in Summary Plot for {model_name} - Year {year}: {e}")

    try:
        top_feature_idx = shap_mean.argmax()
        top_feature = feature_columns[top_feature_idx]
        plt.figure(figsize=(10, 6))
        shap.dependence_plot(top_feature_idx, shap_values, X_test_subset, feature_names=feature_columns, show=False, interaction_index=None)
        plt.title(f"SHAP Dependence Plot for {top_feature} ({model_name}, Year {year})")
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"shap_dependence_{model_name.lower().replace(' ', '_')}_{top_feature}_{year}.png"))
        plt.close('all')
    except Exception as e:
        print(f"Error in Dependence Plot for {model_name} - Year {year}: {e}")

    shap_importance = pd.DataFrame({
        'Feature': feature_columns,
        'SHAP_Value': shap_mean
    }).sort_values(by='SHAP_Value', ascending=False)
    shap_importance.to_csv(os.path.join(output_dir, f"shap_importance_{model_name.lower().replace(' ', '_')}_{year}.csv"), index=False)
    
    shap_importance['Model'] = model_name
    shap_importance['Year'] = year
    return shap_importance

# پردازش داده‌های 2050
dnn_2050 = load_model(os.path.join(output_dir, 'dnn_current_default.h5'), custom_objects={'mse': tf.keras.losses.MeanSquaredError()})
for layer in dnn_2050.layers[:-1]:
    layer.trainable = False
dnn_2050.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())
dnn_2050.fit(X_train_2050, y_train_2050, epochs=50, batch_size=16, validation_split=0.2, callbacks=[early_stopping], verbose=1)

rf_default.fit(X_train_2050, y_train_2050)
xgb_default.fit(X_train_2050, y_train_2050)

rf_grid_2050, rf_bayes_2050, rf_grid_time_2050, rf_bayes_time_2050 = optimize_rf(X_train_2050, y_train_2050)
xgb_grid_2050, xgb_bayes_2050, xgb_grid_time_2050, xgb_bayes_time_2050 = optimize_xgb(X_train_2050, y_train_2050)
dnn_grid_2050, dnn_bayes_2050, dnn_grid_time_2050, dnn_bayes_time_2050 = optimize_dnn(X_train_2050, y_train_2050)

dnn_bayes_2050.save(os.path.join(output_dir, 'dnn_bayes_2050.h5'))

y_pred_rf_default_2050 = rf_default.predict(X_test_2050)
y_pred_xgb_default_2050 = xgb_default.predict(X_test_2050)
y_pred_dnn_default_2050 = dnn_2050.predict(X_test_2050, batch_size=16, verbose=1)
y_pred_rf_grid_2050 = rf_grid_2050.predict(X_test_2050)
y_pred_rf_bayes_2050 = rf_bayes_2050.predict(X_test_2050)
y_pred_xgb_grid_2050 = xgb_grid_2050.predict(X_test_2050)
y_pred_xgb_bayes_2050 = xgb_bayes_2050.predict(X_test_2050)
y_pred_dnn_grid_2050 = dnn_grid_2050.predict(X_test_2050, batch_size=16, verbose=1)
y_pred_dnn_bayes_2050 = dnn_bayes_2050.predict(X_test_2050, batch_size=16, verbose=1)
y_pred_dnn_no_transfer_2050 = dnn_no_transfer_2050.predict(X_test_2050, batch_size=16, verbose=1)

# پردازش داده‌های 2080
dnn_2080 = load_model(os.path.join(output_dir, 'dnn_current_default.h5'), custom_objects={'mse': tf.keras.losses.MeanSquaredError()})
for layer in dnn_2080.layers[:-1]:
    layer.trainable = False
dnn_2080.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())
dnn_2080.fit(X_train_2080, y_train_2080, epochs=50, batch_size=16, validation_split=0.2, callbacks=[early_stopping], verbose=1)

rf_default.fit(X_train_2080, y_train_2080)
xgb_default.fit(X_train_2080, y_train_2080)

rf_grid_2080, rf_bayes_2080, rf_grid_time_2080, rf_bayes_time_2080 = optimize_rf(X_train_2080, y_train_2080)
xgb_grid_2080, xgb_bayes_2080, xgb_grid_time_2080, xgb_bayes_time_2080 = optimize_xgb(X_train_2080, y_train_2080)
dnn_grid_2080, dnn_bayes_2080, dnn_grid_time_2080, dnn_bayes_time_2080 = optimize_dnn(X_train_2080, y_train_2080)

dnn_bayes_2080.save(os.path.join(output_dir, 'dnn_bayes_2080.h5'))

y_pred_rf_default_2080 = rf_default.predict(X_test_2080)
y_pred_xgb_default_2080 = xgb_default.predict(X_test_2080)
y_pred_dnn_default_2080 = dnn_2080.predict(X_test_2080, batch_size=16, verbose=1)
y_pred_rf_grid_2080 = rf_grid_2080.predict(X_test_2080)
y_pred_rf_bayes_2080 = rf_bayes_2080.predict(X_test_2080)
y_pred_xgb_grid_2080 = xgb_grid_2080.predict(X_test_2080)
y_pred_xgb_bayes_2080 = xgb_bayes_2080.predict(X_test_2080)
y_pred_dnn_grid_2080 = dnn_grid_2080.predict(X_test_2080, batch_size=16, verbose=1)
y_pred_dnn_bayes_2080 = dnn_bayes_2080.predict(X_test_2080, batch_size=16, verbose=1)
y_pred_dnn_no_transfer_2080 = dnn_no_transfer_2080.predict(X_test_2080, batch_size=16, verbose=1)

# تابع تحلیل سناریو
def run_scenario(df, X_scaled, model, scenario_name, changes=None):
    df_scenario = df.copy()
    if changes:
        for var, value in changes.items():
            if var in ['P10', 'P11', 'P23', 'P24', 'P25']:
                for col in df_scenario.columns:
                    if col.startswith(f'{var}_'):
                        df_scenario[col] = 0
                dummy_col = f'{var}_{value}'
                if dummy_col in df_scenario.columns:
                    df_scenario[dummy_col] = 1
                else:
                    print(f"Warning: Dummy column {dummy_col} not found in dataframe")
            else:
                df_scenario[var] = value
    df_encoded = df_scenario
    X_scenario = df_encoded.drop(columns=['t1', 't2', 't3', 't4', 't5'])
    X_scenario = X_scenario.reindex(columns=reference_columns, fill_value=0)
    X_scenario_scaled = scaler.transform(X_scenario)
    y_pred = model.predict(X_scenario_scaled) if not isinstance(model, Sequential) else model.predict(X_scenario_scaled, batch_size=16, verbose=1)
    return pd.DataFrame(y_pred, columns=['t1', 't2', 't3', 't4', 't5'])

# تعریف سناریوهای پایه و اضافی
baseline_2050 = y_2050.mean()
baseline_2080 = y_2080.mean()

scenarios_2050 = {
    'Baseline': baseline_2050,
    'High Ventilation': run_scenario(df_2050, X_2050, xgb_bayes_2050, 'High Ventilation', {'P9': 0.0944}).mean(),
    'Efficient HVAC': run_scenario(df_2050, X_2050, xgb_bayes_2050, 'Efficient HVAC', {'P25': 'VRF-DOAS'}).mean(),
    'Optimized Setpoints': run_scenario(df_2050, X_2050, xgb_bayes_2050, 'Optimized Setpoints', {'P1': 22, 'P2': 24}).mean(),
    'Low Infiltration': run_scenario(df_2050, X_2050, xgb_bayes_2050, 'Low Infiltration', {'P3': 0.1}).mean(),
    'Advanced Insulation': run_scenario(df_2050, X_2050, xgb_bayes_2050, 'Advanced Insulation', {
        'P10': 'RoofwithInsulationBatt244mm', 'P23': 'FloorwithInsulationBatt244mm'
    }).mean(),
    'High-Performance Windows': run_scenario(df_2050, X_2050, xgb_bayes_2050, 'High-Performance Windows', {
        'P11': 'QuadrupleLoEFilms(88)3mm/8mmKrypton', 'P24': 'TrpLoEFilm(66)Clr6mm/13mmAir'
    }).mean(),
    'Combined Efficiency': run_scenario(df_2050, X_2050, xgb_bayes_2050, 'Combined Efficiency', {
        'P9': 0.0472, 'P25': 'VRF-DOAS', 'P3': 0.1,
        'P10': 'RoofwithInsulationBatt244mm', 'P23': 'FloorwithInsulationBatt244mm',
        'P11': 'QuadrupleLoEFilms(88)3mm/8mmKrypton', 'P24': 'TrpLoEFilm(66)Clr6mm/13mmAir'
    }).mean()
}

scenarios_2080 = {
    'Baseline': baseline_2080,
    'High Ventilation': run_scenario(df_2080, X_2080, xgb_bayes_2080, 'High Ventilation', {'P9': 0.0944}).mean(),
    'Efficient HVAC': run_scenario(df_2080, X_2080, xgb_bayes_2080, 'Efficient HVAC', {'P25': 'VRF-DOAS'}).mean(),
    'Optimized Setpoints': run_scenario(df_2080, X_2080, xgb_bayes_2080, 'Optimized Setpoints', {'P1': 22, 'P2': 24}).mean(),
    'Low Infiltration': run_scenario(df_2080, X_2080, xgb_bayes_2080, 'Low Infiltration', {'P3': 0.1}).mean(),
    'Advanced Insulation': run_scenario(df_2080, X_2080, xgb_bayes_2080, 'Advanced Insulation', {
        'P10': 'RoofwithInsulationBatt244mm', 'P23': 'FloorwithInsulationBatt244mm'
    }).mean(),
    'High-Performance Windows': run_scenario(df_2080, X_2080, xgb_bayes_2080, 'High-Performance Windows', {
        'P11': 'QuadrupleLoEFilms(88)3mm/8mmKrypton', 'P24': 'TrpLoEFilm(66)Clr6mm/13mmAir'
    }).mean(),
    'Combined Efficiency': run_scenario(df_2080, X_2080, xgb_bayes_2080, 'Combined Efficiency', {
        'P9': 0.0472, 'P25': 'VRF-DOAS', 'P3': 0.1,
        'P10': 'RoofwithInsulationBatt244mm', 'P23': 'FloorwithInsulationBatt244mm',
        'P11': 'QuadrupleLoEFilms(88)3mm/8mmKrypton', 'P24': 'TrpLoEFilm(66)Clr6mm/13mmAir'
    }).mean()
}

# محاسبه عملکرد نسبی سناریوها
scenario_results = []
for year, scenarios in [(2050, scenarios_2050), (2080, scenarios_2080)]:
    for scenario_name, values in scenarios.items():
        if scenario_name != 'Baseline':
            relative_change = ((values - scenarios['Baseline']) / scenarios['Baseline'] * 100).to_dict()
            relative_change['Scenario'] = scenario_name
            relative_change['Year'] = year
            scenario_results.append(relative_change)

# ارزیابی عملکرد مدل‌ها
results = []
for year, X_test, y_test, y_pred_dict in [
    (2050, X_test_2050, y_test_2050, {
        'Random Forest Default': y_pred_rf_default_2050,
        'XGBoost Default': y_pred_xgb_default_2050,
        'DNN Default': y_pred_dnn_default_2050,
        'Random Forest GridSearch': y_pred_rf_grid_2050,
        'Random Forest Bayes': y_pred_rf_bayes_2050,
        'XGBoost GridSearch': y_pred_xgb_grid_2050,
        'XGBoost Bayes': y_pred_xgb_bayes_2050,
        'DNN GridSearch': y_pred_dnn_grid_2050,
        'DNN Bayes': y_pred_dnn_bayes_2050,
        'DNN No Transfer': y_pred_dnn_no_transfer_2050
    }),
    (2080, X_test_2080, y_test_2080, {
        'Random Forest Default': y_pred_rf_default_2080,
        'XGBoost Default': y_pred_xgb_default_2080,
        'DNN Default': y_pred_dnn_default_2080,
        'Random Forest GridSearch': y_pred_rf_grid_2080,
        'Random Forest Bayes': y_pred_rf_bayes_2080,
        'XGBoost GridSearch': y_pred_xgb_grid_2080,
        'XGBoost Bayes': y_pred_xgb_bayes_2080,
        'DNN GridSearch': y_pred_dnn_grid_2080,
        'DNN Bayes': y_pred_dnn_bayes_2080,
        'DNN No Transfer': y_pred_dnn_no_transfer_2080
    })
]:
    for algo_name, y_pred in y_pred_dict.items():
        for i, metric in enumerate(['t1', 't2', 't3', 't4', 't5']):
            metrics = calculate_metrics(y_test[metric], y_pred[:, i], metric)
            metrics['Algorithm'] = algo_name
            metrics['Year'] = year
            results.append(metrics)
            print(f"{algo_name} ({year}, {metric}) - RMSE: {metrics['RMSE']:.2f}, R²: {metrics['R²']:.2f}, MAPE: {metrics['MAPE']:.2f}%")

# تحلیل بوت‌استرپ
bootstrap_results = []
for year, X_test, y_test, model_dict in [
    (2050, X_test_2050, y_test_2050, {
        'Random Forest Default': rf_default, 'XGBoost Default': xgb_default, 'DNN Default': dnn_2050,
        'Random Forest GridSearch': rf_grid_2050, 'Random Forest Bayes': rf_bayes_2050,
        'XGBoost GridSearch': xgb_grid_2050, 'XGBoost Bayes': xgb_bayes_2050,
        'DNN GridSearch': dnn_grid_2050, 'DNN Bayes': dnn_bayes_2050,
        'DNN No Transfer': dnn_no_transfer_2050
    }),
    (2080, X_test_2080, y_test_2080, {
        'Random Forest Default': rf_default, 'XGBoost Default': xgb_default, 'DNN Default': dnn_2080,
        'Random Forest GridSearch': rf_grid_2080, 'Random Forest Bayes': rf_bayes_2080,
        'XGBoost GridSearch': xgb_grid_2080, 'XGBoost Bayes': xgb_bayes_2080,
        'DNN GridSearch': dnn_grid_2080, 'DNN Bayes': dnn_bayes_2080,
        'DNN No Transfer': dnn_no_transfer_2080
    })
]:
    for algo_name, model in model_dict.items():
        bootstrap_results.append(summarize_bootstrap(bootstrap_metrics(model, X_test, y_test, algo_name), algo_name, year))

# تحلیل SHAP
shap_results = []
for year, X_test, model_dict in [
    (2050, X_test_2050, {
        'Random Forest Bayes': rf_bayes_2050,
        'XGBoost Bayes': xgb_bayes_2050,
        'DNN Bayes': dnn_bayes_2050
    }),
    (2080, X_test_2080, {
        'Random Forest Bayes': rf_bayes_2080,
        'XGBoost Bayes': xgb_bayes_2080,
        'DNN Bayes': dnn_bayes_2080
    })
]:
    for model_name, model in model_dict.items():
        shap_importance = run_shap_analysis(model, X_test, feature_names, model_name, year)
        if not shap_importance.empty:
            shap_results.append(shap_importance)

# ذخیره نتایج
results_df = pd.DataFrame(results)
results_df.to_csv(os.path.join(output_dir, 'algorithm_performance_2050_2080.csv'), index=False)
bootstrap_summary = pd.concat(bootstrap_results, ignore_index=True)
bootstrap_summary.to_csv(os.path.join(output_dir, 'bootstrap_summary_2050_2080.csv'))
scenario_results_df = pd.DataFrame(scenario_results)
scenario_results_df.to_csv(os.path.join(output_dir, 'scenario_results_2050_2080.csv'), index=False)
if shap_results:
    shap_results_df = pd.concat(shap_results, ignore_index=True)
    shap_results_df.to_csv(os.path.join(output_dir, 'shap_importance_all.csv'), index=False)

# 1. نمودارهای پراکندگی برای مدل‌های پیش‌فرض
for year, y_test, y_pred_dict in [
    (2050, y_test_2050, {
        'Random Forest': y_pred_rf_default_2050,
        'XGBoost': y_pred_xgb_default_2050,
        'DNN': y_pred_dnn_default_2050
    }),
    (2080, y_test_2080, {
        'Random Forest': y_pred_rf_default_2080,
        'XGBoost': y_pred_xgb_default_2080,
        'DNN': y_pred_dnn_default_2080
    })
]:
    for algo_name, y_pred in y_pred_dict.items():
        plt.figure(figsize=(15, 10))
        for i, metric in enumerate(['t1', 't2', 't3', 't4', 't5']):
            plt.subplot(2, 3, i+1)
            # نقاط واقعی (Actual)
            plt.scatter(y_test[metric], y_test[metric], c='blue', alpha=0.5, label='Actual')
            # نقاط پیش‌بینی‌شده (Predicted)
            plt.scatter(y_test[metric], y_pred[:, i], c='orange', alpha=0.5, label='Predicted')
            plt.plot([y_test[metric].min(), y_test[metric].max()], [y_test[metric].min(), y_test[metric].max()], 'k--')
            plt.xlabel(f'Actual {metric}')
            plt.ylabel(f'Predicted {metric}')
            plt.title(f'{metric} - {algo_name}')
            plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'scatter_default_{algo_name}_{year}.png'))
        plt.close('all')

# 2. مقایسه RMSE، R² و MAPE برای مدل‌های پیش‌فرض
for year in [2050, 2080]:
    plt.figure(figsize=(18, 5))
    plt.subplot(1, 3, 1)
    for algo in ['Random Forest Default', 'XGBoost Default', 'DNN Default']:
        algo_data = results_df[(results_df['Algorithm'] == algo) & (results_df['Year'] == year)]
        plt.bar(algo_data['Metric'], algo_data['RMSE'], label=algo, alpha=0.3)
    plt.title(f'Test RMSE - Year {year}')
    plt.xlabel('Metric')
    plt.ylabel('RMSE')
    plt.legend()
    plt.subplot(1, 3, 2)
    for algo in ['Random Forest Default', 'XGBoost Default', 'DNN Default']:
        algo_data = results_df[(results_df['Algorithm'] == algo) & (results_df['Year'] == year)]
        plt.bar(algo_data['Metric'], algo_data['R²'], label=algo, alpha=0.3)
    plt.title(f'Test R² - Year {year}')
    plt.xlabel('Metric')
    plt.ylabel('R²')
    plt.legend()
    plt.subplot(1, 3, 3)
    for algo in ['Random Forest Default', 'XGBoost Default', 'DNN Default']:
        algo_data = results_df[(results_df['Algorithm'] == algo) & (results_df['Year'] == year)]
        plt.bar(algo_data['Metric'], algo_data['MAPE'], label=algo, alpha=0.3)
    plt.title(f'Test MAPE - Year {year}')
    plt.xlabel('Metric')
    plt.ylabel('MAPE (%)')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'comparison_default_{year}.png'))
    plt.close('all')

# 3. مقایسه RMSE، R² و MAPE برای مدل‌های بهینه‌شده
for year in [2050, 2080]:
    plt.figure(figsize=(18, 5))
    plt.subplot(1, 3, 1)
    for algo in ['Random Forest GridSearch', 'Random Forest Bayes', 'XGBoost GridSearch', 'XGBoost Bayes', 'DNN GridSearch', 'DNN Bayes', 'DNN No Transfer']:
        algo_data = results_df[(results_df['Algorithm'] == algo) & (results_df['Year'] == year)]
        plt.bar(algo_data['Metric'], algo_data['RMSE'], label=algo, alpha=0.3)
    plt.title(f'Test RMSE (Optimized) - Year {year}')
    plt.xlabel('Metric')
    plt.ylabel('RMSE')
    plt.legend()
    plt.subplot(1, 3, 2)
    for algo in ['Random Forest GridSearch', 'Random Forest Bayes', 'XGBoost GridSearch', 'XGBoost Bayes', 'DNN GridSearch', 'DNN Bayes', 'DNN No Transfer']:
        algo_data = results_df[(results_df['Algorithm'] == algo) & (results_df['Year'] == year)]
        plt.bar(algo_data['Metric'], algo_data['R²'], label=algo, alpha=0.3)
    plt.title(f'Test R² (Optimized) - Year {year}')
    plt.xlabel('Metric')
    plt.ylabel('R²')
    plt.legend()
    plt.subplot(1, 3, 3)
    for algo in ['Random Forest GridSearch', 'Random Forest Bayes', 'XGBoost GridSearch', 'XGBoost Bayes', 'DNN GridSearch', 'DNN Bayes', 'DNN No Transfer']:
        algo_data = results_df[(results_df['Algorithm'] == algo) & (results_df['Year'] == year)]
        plt.bar(algo_data['Metric'], algo_data['MAPE'], label=algo, alpha=0.3)
    plt.title(f'Test MAPE (Optimized) - Year {year}')
    plt.xlabel('Metric')
    plt.ylabel('MAPE (%)')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'comparison_optimized_{year}.png'))
    plt.close('all')

# 4. نمودارهای پراکندگی برای مدل‌های بهینه‌شده
for year, y_test, y_pred_dict in [
    (2050, y_test_2050, {
        'Random Forest GridSearch': y_pred_rf_grid_2050,
        'Random Forest Bayes': y_pred_rf_bayes_2050,
        'XGBoost GridSearch': y_pred_xgb_grid_2050,
        'XGBoost Bayes': y_pred_xgb_bayes_2050,
        'DNN GridSearch': y_pred_dnn_grid_2050,
        'DNN Bayes': y_pred_dnn_bayes_2050,
        'DNN No Transfer': y_pred_dnn_no_transfer_2050
    }),
    (2080, y_test_2080, {
        'Random Forest GridSearch': y_pred_rf_grid_2080,
        'Random Forest Bayes': y_pred_rf_bayes_2080,
        'XGBoost GridSearch': y_pred_xgb_grid_2080,
        'XGBoost Bayes': y_pred_xgb_bayes_2080,
        'DNN GridSearch': y_pred_dnn_grid_2080,
        'DNN Bayes': y_pred_dnn_bayes_2080,
        'DNN No Transfer': y_pred_dnn_no_transfer_2080
    })
]:
    for algo_name, y_pred in y_pred_dict.items():
        plt.figure(figsize=(15, 10))
        for i, metric in enumerate(['t1', 't2', 't3', 't4', 't5']):
            plt.subplot(2, 3, i+1)
            # نقاط واقعی (Actual)
            plt.scatter(y_test[metric], y_test[metric], c='blue', alpha=0.5, label='Actual')
            # نقاط پیش‌بینی‌شده (Predicted)
            plt.scatter(y_test[metric], y_pred[:, i], c='orange', alpha=0.5, label='Predicted')
            plt.plot([y_test[metric].min(), y_test[metric].max()], [y_test[metric].min(), y_test[metric].max()], 'k--')
            plt.xlabel(f'Actual {metric}')
            plt.ylabel(f'Predicted {metric}')
            plt.title(f'{metric} - {algo_name}')
            plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'scatter_optimized_{algo_name}_{year}.png'))
        plt.close('all')

# 5. نمودار کارایی محاسباتی
times = {
    'RF GridSearch 2050': rf_grid_time_2050,
    'RF Bayes 2050': rf_bayes_time_2050,
    'XGB GridSearch 2050': xgb_grid_time_2050,
    'XGB Bayes 2050': xgb_bayes_time_2050,
    'DNN GridSearch 2050': dnn_grid_time_2050,
    'DNN Bayes 2050': dnn_bayes_time_2050,
    'RF GridSearch 2080': rf_grid_time_2080,
    'RF Bayes 2080': rf_bayes_time_2080,
    'XGB GridSearch 2080': xgb_grid_time_2080,
    'XGB Bayes 2080': xgb_bayes_time_2080,
    'DNN GridSearch 2080': dnn_grid_time_2080,
    'DNN Bayes 2080': dnn_bayes_time_2080
}
plt.figure(figsize=(10, 6))
plt.bar(times.keys(), times.values())
plt.xticks(rotation=45)
plt.title('Computational Efficiency of GridSearchCV vs. Bayesian Optimization')
plt.xlabel('Method')
plt.ylabel('Time (seconds)')
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'computational_efficiency.png'))
plt.close('all')

# 6. نمودارهای توزیع باقی‌مانده‌ها (ویولن)
for year, y_test, y_pred_dict in [
    (2050, y_test_2050, {
        'Random Forest Default': y_pred_rf_default_2050,
        'XGBoost Default': y_pred_xgb_default_2050,
        'DNN Default': y_pred_dnn_default_2050,
        'Random Forest Bayes': y_pred_rf_bayes_2050,
        'XGBoost Bayes': y_pred_xgb_bayes_2050,
        'DNN Bayes': y_pred_dnn_bayes_2050,
        'DNN No Transfer': y_pred_dnn_no_transfer_2050
    }),
    (2080, y_test_2080, {
        'Random Forest Default': y_pred_rf_default_2080,
        'XGBoost Default': y_pred_xgb_default_2080,
        'DNN Default': y_pred_dnn_default_2080,
        'Random Forest Bayes': y_pred_rf_bayes_2080,
        'XGBoost Bayes': y_pred_xgb_bayes_2080,
        'DNN Bayes': y_pred_dnn_bayes_2080,
        'DNN No Transfer': y_pred_dnn_no_transfer_2080
    })
]:
    plt.figure(figsize=(15, 10))
    for i, metric in enumerate(['t1', 't2', 't3', 't4', 't5']):
        residuals = []
        labels = []
        for algo_name, y_pred in y_pred_dict.items():
            residuals.append(y_test[metric] - y_pred[:, i])
            labels.append(algo_name)
        plt.subplot(2, 3, i+1)
        sns.violinplot(data=residuals, orient='v')
        plt.xticks(range(len(labels)), labels, rotation=45)
        plt.title(f'Residuals for {metric} - Year {year}')
        plt.ylabel('Residual')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'residuals_violin_{year}.png'))
    plt.close('all')

# 7. نمودارهای تعادل (Trade-off)
for year, scenarios in [(2050, scenarios_2050), (2080, scenarios_2080)]:
    plt.figure(figsize=(10, 6))
    for scenario_name, values in scenarios.items():
        if scenario_name != 'Baseline':
            plt.scatter(values['t1'], values['t3'], label=scenario_name, s=100)
    plt.scatter(scenarios['Baseline']['t1'], scenarios['Baseline']['t3'], label='Baseline', s=100, marker='x')
    plt.xlabel('Primary Energy Consumption (t1, kWh)')
    plt.ylabel('Indoor Air Quality (t3, ppm)')
    plt.title(f'Trade-off: t1 vs t3 - Year {year}')
    plt.legend()
    plt.savefig(os.path.join(output_dir, f'tradeoff_t1_t3_{year}.png'))
    plt.close('all')

# 8. نمودارهای بوت‌استرپ
for year in [2050, 2080]:
    plt.figure(figsize=(18, 5))
    plt.subplot(1, 3, 1)
    for algo in ['Random Forest Bayes', 'XGBoost Bayes', 'DNN Bayes', 'DNN No Transfer']:
        algo_data = bootstrap_summary[(bootstrap_summary['Algorithm'] == algo) & (bootstrap_summary['Year'] == year)]
        plt.errorbar(algo_data['Metric'], algo_data['RMSE']['mean'], 
                     yerr=[algo_data['RMSE']['mean'] - algo_data['RMSE']['<lambda_0>'], 
                           algo_data['RMSE']['<lambda_1>'] - algo_data['RMSE']['mean']], 
                     label=algo, capsize=5)
    plt.title(f'Bootstrap RMSE - Year {year}')
    plt.xlabel('Metric')
    plt.ylabel('RMSE')
    plt.legend()
    plt.subplot(1, 3, 2)
    for algo in ['Random Forest Bayes', 'XGBoost Bayes', 'DNN Bayes', 'DNN No Transfer']:
        algo_data = bootstrap_summary[(bootstrap_summary['Algorithm'] == algo) & (bootstrap_summary['Year'] == year)]
        plt.errorbar(algo_data['Metric'], algo_data['R²']['mean'], 
                     yerr=[algo_data['R²']['mean'] - algo_data['R²']['<lambda_0>'], 
                           algo_data['R²']['<lambda_1>'] - algo_data['R²']['mean']], 
                     label=algo, capsize=5)
    plt.title(f'Bootstrap R² - Year {year}')
    plt.xlabel('Metric')
    plt.ylabel('R²')
    plt.legend()
    plt.subplot(1, 3, 3)
    for algo in ['Random Forest Bayes', 'XGBoost Bayes', 'DNN Bayes', 'DNN No Transfer']:
        algo_data = bootstrap_summary[(bootstrap_summary['Algorithm'] == algo) & (bootstrap_summary['Year'] == year)]
        plt.errorbar(algo_data['Metric'], algo_data['MAPE']['mean'], 
                     yerr=[algo_data['MAPE']['mean'] - algo_data['MAPE']['<lambda_0>'], 
                           algo_data['MAPE']['<lambda_1>'] - algo_data['MAPE']['mean']], 
                     label=algo, capsize=5)
    plt.title(f'Bootstrap MAPE - Year {year}')
    plt.xlabel('Metric')
    plt.ylabel('MAPE (%)')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'bootstrap_{year}.png'))
    plt.close('all')

# 9. عملکرد سناریوها نسبت به پایه
for year in [2050, 2080]:
    plt.figure(figsize=(15, 6))
    for scenario in ['High Ventilation', 'Efficient HVAC', 'Optimized Setpoints', 'Low Infiltration', 
                     'Advanced Insulation', 'High-Performance Windows', 'Combined Efficiency']:
        scenario_data = scenario_results_df[(scenario_results_df['Scenario'] == scenario) & (scenario_results_df['Year'] == year)]
        plt.bar([f'{metric}_{scenario}' for metric in ['t1', 't2', 't3', 't4', 't5']], 
                [scenario_data[metric].values[0] for metric in ['t1', 't2', 't3', 't4', 't5']], 
                label=scenario, alpha=0.5)
    plt.title(f'Scenario Performance Relative to Baseline - Year {year}')
    plt.xlabel('Metric')
    plt.ylabel('Relative Change (%)')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'scenario_performance_{year}.png'))
    plt.close('all')

print(f"All results and plots saved to: {output_dir}")

C:\Users\user\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


All datasets loaded successfully!
Columns in df_current: ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10_RoofwithInsulationBatt154mm', 'P10_RoofwithInsulationBatt244mm', 'P10_RoofwithInsulationBatt89mm', 'P10_RoofwithInsulationBoard25mm', 'P10_RoofwithInsulationBoard50mm', 'P10_RoofwithInsulationBoard75mm', 'P10_RoofwithInsulationCellularGlass25mm', 'P10_RoofwithInsulationCellularGlass50mm', 'P10_RoofwithInsulationCellularGlass75mm', 'P10_RoofwithInsulationExpandedPerlite25mm', 'P10_RoofwithInsulationExpandedrubber(rigid)25mm', 'P10_RoofwithInsulationGlassfiber25mm', 'P10_RoofwithInsulationGlassfiber50mm', 'P10_RoofwithInsulationGlassfiber75mm', 'P10_RoofwithInsulationXPS25mm', 'P10_RoofwithInsulationXPS50mm', 'P10_RoofwithInsulationXPS75mm', 'P10_RoofwithVegetation', 'P11_DblBronze6mm/13mmAir', 'P11_DblClr6mm/13mmAir', 'P11_DblElecAbsBleached6mm/13mmAir', 'P11_DblElecAbsColored6mm/13mmAir', 'P11_DblElecRefBleached6mm/13mmAir', 'P11_DblElecRefColored6mm/13mmAir', 'P11_DblGr

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StatelessRandomGetKeyCounter in device /job:localhost/replica:0/task:0/device:CPU:0
Executin

Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV3 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Epoch 1/50
Executing op ReadV

Executing op __inference_train_function_1179 in device /job:localhost/replica:0/task:0/device:CPU:0
13/51 [======>.......................] - ETA: 0s - loss: 618740736.0000Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_function_1179 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_function_1179 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_train_function_1179 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localh

...

Executing op __inference_predict_function_39985 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_predict_function_39985 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_predict_function_39985 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_predict_function_39985 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in 

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Shape in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Shape in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/

Executing op Shape in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Shape in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Shape in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replic

Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_predict_function_39985 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_predict_function_39985 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_predict_function_39985 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_predict_function_39985 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in 

Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Shape in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Shape in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op StridedSlice in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Shape in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op _EagerConst in device /job:localhost/repli

In [2]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

     ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
     --------------------------------------- 0.0/10.8 MB 660.6 kB/s eta 0:00:17
     --------------------------------------- 0.0/10.8 MB 281.8 kB/s eta 0:00:39
     --------------------------------------- 0.0/10.8 MB 281.8 kB/s eta 0:00:39
     --------------------------------------- 0.1/10.8 MB 386.4 kB/s eta 0:00:28
     --------------------------------------- 0.1/10.8 MB 386.4 kB/s eta 0:00:28
      -------------------------------------- 0.2/10.8 MB 403.5 kB/s eta 0:00:27
      -------------------------------------- 0.2/10.8 MB 393.8 kB/s eta 0:00:27
     - ------------------------------------- 0.3/10.8 MB 559.2 kB/s eta 0:00:19
     - ------------------------------------- 0.4/10.8 MB 604.4 kB/s eta 0:00:18
     - ------------------------------------- 0.5/10.8 MB 796.1 k


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: C:\Program Files\Python38\python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
print(pd.__version__)

2.0.3


In [2]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/9.3 MB 320.0 kB/s eta 0:00:29
     ---------------------------------------- 0.0/9.3 MB 279.3 kB/s eta 0:00:34
     ---------------------------------------- 0.1/9.3 MB 435.7 kB/s eta 0:00:22
      --------------------------------------- 0.1/9.3 MB 450.6 kB/s eta 0:00:21
      --------------------------------------- 0.1/9.3 MB 448.2 kB/s eta 0:00:21
      --------------------------------------- 0.2/9.3 MB 476.3 kB/s eta 0:00:20
      --------------------------------------- 0.2/9.3 MB 476.3 kB/s eta 0:00:20
      --------------------------------------- 0.2/9.3 MB 476.3 kB/s eta 0:00:20
     - -------------------------------------- 0.3/9.3 MB 654.5 kB/s eta 0:00:14
     - -------------------------------------- 0.4/9.3 MB 714.4 kB/s eta 0:00:13
     - -------------------------------------- 0.4/9


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: C:\Program Files\Python38\python.exe -m pip install --upgrade pip


In [4]:
import numpy as np
import sklearn
print("NumPy version:", np.__version__)
print("Scikit-learn version:", sklearn.__version__)

NumPy version: 1.24.4
Scikit-learn version: 1.3.2


In [5]:
pip install xgboost

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Using cached xgboost-2.1.4-py3-none-win_amd64.whl (124.9 MB)



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: C:\Program Files\Python38\python.exe -m pip install --upgrade pip


In [3]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.13.1-cp38-cp38-win_amd64.whl (1.9 kB)
  Using cached tensorflow-2.13.0-cp38-cp38-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.13.0-cp38-cp38-win_amd64.whl (276.5 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl (135 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached grpcio-1.70.0-cp38-cp38-win_amd64.whl (4.3 MB)
  Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.11.0-cp38-cp38-win_amd64.whl (3.0 MB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached Markdown-3.7-py3-none-any.whl (106 kB)
  Using cached requests-2.32.4-py3-none-any.whl (64 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Using cached google_auth-2.43.0-py2.py3-none-any.whl (223 kB)
  Using 


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: C:\Program Files\Python38\python.exe -m pip install --upgrade pip


In [5]:
pip install bayesian-optimization shap

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: C:\Program Files\Python38\python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/450.3 kB ? eta -:--:--
      --------------------------------------- 10.2/450.3 kB ? eta -:--:--
     -- ---------------------------------- 30.7/450.3 kB 640.0 kB/s eta 0:00:01
     ----- ------------------------------- 71.7/450.3 kB 491.5 kB/s eta 0:00:01
     ------- ----------------------------- 92.2/450.3 kB 581.0 kB/s eta 0:00:01
     ------- ----------------------------- 92.2/450.3 kB 581.0 kB/s eta 0:00:01
     ------- ----------------------------- 92.2/450.3 kB 581.0 kB/s eta 0:00:01
     ------------ ----------------------- 153.6/450.3 kB 458.0 kB/s eta 0:00:01
     ------------ ----------------------- 153.6/450.3 kB 458.0 kB/s eta 0:00:01
     ------------- ---------------------- 174.1/450.3 kB 402.6 kB/s eta 0:00:01
     ------------------ ----------------- 235.5/450.3 kB 496.2 kB/s eta 0:00:01
     -------------------- --------------- 256.0/450.3 kB 524.0 kB/s eta 0:00:01
     --------------------- -------------- 266.2/450.